# Imports

In [85]:
import pandas as pd; import seaborn as sb; import warnings; import scipy; import re; 
import os; from analysis_helpers import *; import itertools; from scipy import stats
import random; import pandas as pd; import numpy as np; from sklearn import datasets, linear_model; 
from sklearn.linear_model import LinearRegression; import statsmodels.api as sm
from scipy import stats; from itertools import groupby; from operator import itemgetter; import random
warnings.filterwarnings('ignore')
%matplotlib inline

# Load Data

In [86]:
exp1 = pd.DataFrame.from_csv('../parsed_data/behavioral_data_sustained.csv')
exp2 = pd.DataFrame.from_csv('../parsed_data/behavioral_data_variable.csv' )

# label rows by trial number
for exp in [exp1, exp2]:

    # Number all presentation and memory trials 
    exp.loc[exp['Trial Type']=='Memory','Trial'] = list(range(0,40))*30*8
    exp.loc[exp['Trial Type']=='Presentation','Trial'] = list(range(0,10))*30*8

In [87]:
window_length=5

# Updated Sliding Window Function

In [88]:
def apply_window(combo, window_length):
    '''
    input:  dataframe of behavioral data from an entire experiment
    output: dataframe of same shape where raw values have been replaced by rolling window mean
    '''

    # select data from memory runs
    data = combo[combo['Trial Type']=='Memory'][['Attention Level','Familiarity Rating','Trial','Subject','Run']]

    # re-structure the data - each row is a trial, each column is an attn level
    df = data.pivot_table(index=['Subject','Trial'], columns='Attention Level', values='Familiarity Rating')

    # apply rolling window, for each run in each subject
    window_data = df.groupby(['Subject']).apply(lambda x: x.rolling(window_length, min_periods=1, center=True).mean())

    return(window_data)

In [106]:
exp1_window = apply_window(exp1, 5)
exp1_window.head(n=10)

Attention Level  Category      Full  None     Novel      Side
Subject Trial                                                
0       0.0      1.000000  2.000000  1.00  1.749206  1.000000
        1.0      1.333333  2.333333  1.00  1.611905  1.000000
        2.0      1.333333  2.375000  1.00  1.889524  1.111111
        3.0      1.666667  2.166667  1.00  1.889524  1.111111
        4.0      1.583333  2.166667  1.00  1.746667  1.083333
        5.0      1.583333  2.333333  1.25  1.506667  1.266667
        6.0      1.833333  2.000000  1.25  1.700000  1.333333
        7.0      1.666667  1.500000  1.75  1.500000  1.333333
        8.0      1.666667  1.500000  1.50  1.600000  1.500000
        9.0      1.750000  1.375000  1.50  1.700000  2.000000

# Verify correct Output

Average across all runs for each subject, take the mean(s) of these values and show they are the same as the sliding window values. 

In [108]:
# select data from memory runs
data = exp1[exp1['Trial Type']=='Memory'][['Attention Level','Familiarity Rating','Trial','Subject','Run']]

# re-structure the data - each row is a trial, each column is an attn level
df = data.pivot_table(index=['Subject','Trial'], columns='Attention Level', values='Familiarity Rating')

df.groupby(['Subject','Trial']).mean().head(n=10)

Attention Level  Category  Full  None     Novel      Side
Subject Trial                                            
0       0.0      1.000000   3.0   1.0  1.333333  1.000000
        1.0           NaN   NaN   1.0  1.714286       NaN
        2.0      1.000000   1.0   1.0  2.200000       NaN
        3.0      2.000000   3.0   NaN  1.200000  1.000000
        4.0           NaN   2.5   1.0  3.000000  1.333333
        5.0      2.000000   NaN   NaN  1.333333  1.000000
        6.0      1.333333   NaN   NaN  1.000000  1.000000
        7.0      1.000000   1.5   1.5  1.000000  2.000000
        8.0      3.000000   2.0   NaN  2.166667       NaN
        9.0      1.000000   1.0   2.0  2.000000       NaN